# Computational Theory Problems

# Introduction
This notebook contains the primary implementation of the SHA-256 hashing algorithm specified in the [Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf). This is a cryptographic hash function that produces a 256-bit (32-byte) hash value, used to verify data integrity and password hashing.

The notebook is structured into 5 different problems that reference different parts of the SHA-256 algorithm:
1. **Problem 1**: Implementing the core bitwise functions used in SHA-256
2. **Problem 2**: Calculating the K constants from the cube roots of prime numbers
3. **Problem 3**: Creating a message padding and block parsing function
4. **Problem 4**: Implementing the SHA-256 hash computation
5. **Problem 5**: Applying the implementation to crack password hashes

Each problem builds upon the previous ones, culminating in a working SHA-256 implementation.

In [1]:
# Allowed Imports according to requirements.txt
import numpy as np
# Ignore overfow warnings for uint32 operations
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Problem 1: Binary Words and Operations
## Overview
Following the Secure Hash Standard PDF, we implement seven bitwise functions that are fundamental to the SHA hashing algorithms.

### Implementation Steps
1. Add the helper functions - before adding the main 7 bitwise functions, these are utility functions for rotating and shifting bits:
   - `rotr(x, n)` - Rotate right
   - `rotl(x, n)` - Rotate left
   - `shr(x, n)` - Right shift with zeros
 
2. Implement the 7 main bitwise functions directly from the NIST FIPS PDF specification:
   - `Parity(x, y, z)` - Parity operation
   - `Ch(x, y, z)` - Choose operation
   - `Maj(x, y, z)` - Majority operation
   - `Sigma0(x) Σ₀` & `Sigma1(x) Σ₁` - Uppercase sigma functions (rotation based)
   - `sigma0(x) σ₀` & `sigma1(x) σ₁` - Lowercase sigma functions (shift based)
  
3. Verify Implementation - test each function to ensure that they work as expected.
   - Checks against known values
   - Verify that the bitwise behaviour works as expected
   - Compare results with specific examples

## Approach
### Use of numpy.uint32
We use `np.uint32()` to:
- Enforce 32-bit boundaries to the outputs
- This ensures a consistent behaviour across different systems
- Uses the NIST specification assuming the use of 32-bit words

In Python, integers can get very large so we need to make sure that all outputs are configured to be 32-bit unsigned integers so we can apply the SHA rules and specifications.

### Rotation & Shifting
We have two types of bit operations in these functions:

**Rotation (ROTR, ROTL)**: 
- Bits wrap around in a circle
- No bits are lost in the operation
- Used when we need to preserve all information while mixing bits

**Shifting (SHR)**:
- Bits are dropped off one end
- Information is intentionally discarded
- Used when we want diffusion (spreading the change across bits)

In SHA-256:
- Rotations appear in Sigma functions (uppercase - need to preserve all bits)
- Shifts appear in sigma functions (lowercase - intentional bit loss for diffusion)

### The Seven Bitwise Functions

We implement exactly 7 functions because they provide:
- **Diffusion**: Changing one input bit affects many output bits (security)
- **Confusion**: Input-output relationship is complex and non-linear (security)
- **Speed**: Bitwise operations are extremely fast on all processors
- **Cryptographic strength**: They prevent certain types of attacks

These functions come directly from Section 4.1.2.

## Symbols and Operations:

### `& (Bitwise AND operation)`
Returns 1 only when both corresponding bits are 1. If either bit has a 0, the result will be 0.

2-bit Example: 1100 & 1010 = 1000

3-bit Example: 1100 & 1010 & 1110 = 1000

### `| (Bitwise OR operation)`
Returns 1 only when at least the corresponding bits are 1. It will return 0 when both or all bits are 0.

2-bit Example: 1100 | 1010 = 1110

3-bit Example: 1100 | 1010 | 1110 = 1110

### `^ (Bitwise XOR operation - exclusive-OR)`
Returns 1 when an odd number of corresponding bits are 1. It will return 0 when an even number of bits are 1.

2-bit Example: 1100 ^ 1010 = 0110

3-bit Example: 1100 ^ 1010 ^ 1110 = 0110

### `~ (Bitwise complement operation)`
Inverts all bits - changes all the 0s to 1s and vice versa. 

2-bit Example: ~1100 = 0011

3-bit Example: ~1100 & ~1010 & ~1110 = 0001

### `<< (Left-Shift operation)`
Ignores the left-most n bits and pads the result with n zeros on the right. It essentially multiplies the bits by 2^n.

2-bit Example: 1100 << 2 = 0000

3-bit Example: 0011 << 3 = 1000

### `>> (Right-Shift operation)`
Ignores the right-most n bits and pads the result with n zeros on the left. It essentially divides the bits by 2^n.

2-bit Example: 1100 >> 2 = 0011

3-bit Example: 1000 >> 3 = 0001

In [2]:
# Helper functions
# Right rotate operation function 
def rotr(x, n):
    """
    Rotate right (circular right shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    # X is converted to uint32 to ensure proper bitwise operations
    x = np.uint32(x)
    # The rotation is performed by shifting right and left, then combining with bitwise OR
    return np.uint32((x >> n) | (x << (32 - n)))

# Left rotate operation function
def rotl(x, n):
    """
    Rotate left (circular left shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    # X is converted to uint32 to ensure proper bitwise operations
    x = np.uint32(x)
    # The rotation is performed by shifting left and right, then combining with bitwise OR
    return np.uint32((x << n) | (x >> (32 - n)))

# Right shift operation function
def shr(x, n):
    """
    A right shift operation only shifts with zeros, no rotating.

    Parameters:
        x (int): The 32-bit integer to shift.
        n (int): The number of bits/positions to shift.

    Returns:
        int: The shifted integer as a 32-bit integer.
    """
    # X is converted to uint32 to ensure proper bitwise operations
    x = np.uint32(x)
    # The right shift is performed by shifting right n bits
    return np.uint32(x >> n)

In [3]:
# Parity(x, y, z) function

def Parity(x, y, z):
    """
    This function calculates the parity of three 32-bit unsigned integers.

    It will return the result of the bitwise XOR operation on the three inputs.

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The parity result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Calculate the parity by performing bitwise XOR on the three inputs
    return np.uint32(x ^ y ^ z)

In [4]:
# Ch(x, y, z) Function
def Ch(x, y, z):
   """
   This function calculates the choice function of three 32-bit unsigned integers.

   It will return the result of the bitwise operation: (x AND y) XOR (NOT x AND z).

   Parameters:
         x (int): First 32-bit unsigned integer.
         y (int): Second 32-bit unsigned integer.
         z (int): Third 32-bit unsigned integer.
   Returns:
         int: The choice result as a 32-bit unsigned integer.
   """
   x = np.uint32(x) 
   y = np.uint32(y)
   z = np.uint32(z)
   
   # Calculate the choice function performing bitwise AND, NOT, and XOR operations
   return np.uint32((x & y) ^ (~x & z))

In [5]:
# Maj(x, y, z) Function
def Maj(x, y, z):
    """
    This function calculates the majority function of three 32-bit unsigned integers.

    It will return the result of the bitwise operation: (x AND y) XOR (x AND z) XOR (y AND z).

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The majority result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Calculate the majority function by performing bitwise AND and XOR operations
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [6]:
# Sigma0(x) Function E0
def Sigma0(x):
    """
    This function calculates the Sigma0 (E0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 2) XOR ROTR(x, 13) XOR ROTR(x, 22).
    ROTR is the right rotate operation.

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Calculate the Sigma0 function using right rotations and bitwise XOR
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


In [7]:
# Sigma1(x) Function E1
def Sigma1(x):
    """
    This function calculates the Sigma1 (E1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 6) XOR ROTR(x, 11) XOR ROTR(x, 25).
    ROTR is the right rotate operation.

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Calculate the Sigma1 function by performing bitwise ROTR operations
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

In [8]:
# Sigma0(x) Function O0
def sigma0(x):
    """
    This function calculates the sigma0 (o0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 7) XOR ROTR(x, 18) XOR SHR(x, 3).
    ROTR is the right rotate operation.
    SHR is the right shift operation.

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Calculate the sigma0 function by performing bitwise ROTR and SHR operations
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)) 

In [9]:
# Sigma1(x) Function O1
def sigma1(x):
    """
    This function calculates the sigma1 (o1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 17) XOR ROTR(x, 19) XOR SHR(x, 10).
    ROTR is the right rotate operation.
    SHR is the right shift operation.

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Calculate the sigma1 function by performing bitwise ROTR and SHR operations
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

## Testing the functions in Problem 1

In [10]:
# Test Parity Function with hexadecimal inputs
print("Testing Parity Function with hexadecimal inputs:")
# Test with hexadecimal 0xF which is 15 in decimal
result = Parity(0xF, 0xF, 0xF)
# Expected result should be 0xF as 0xF ^ 0xF ^ 0xF = 0xF
print(f"Expected: 0xf")
# Output the result in hexadecimal format
print(f"Result: {hex(result)}" + "\n")

# Test 2
# All bits set to 1 for first two inputs, all bits set to 0 for third input
print("Test 2")
result = Parity(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
print(f"Expected: 0x0")
print(f"Result: {hex(result)}" + "\n")

# Test 3
# Alternating bits for first two inputs, all bits set to 0 for third input
print("Test 3")
result = Parity(0xAAAAAAAA, 0x55555555, 0x00000000)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")


Testing Parity Function with hexadecimal inputs:
Expected: 0xf
Result: 0xf

Test 2
Expected: 0x0
Result: 0x0

Test 3
Expected: 0xffffffff
Result: 0xffffffff



In [11]:
# Choice Function Test with hexadecimal inputs
print("\nTesting Choice Function with hexadecimal inputs:")
# Test with hexadecimal 0xF which is 15 in decimal
result = Ch(0xF, 0x0, 0xF)
# Expected result should be 0x0 as (0xF & 0x0) ^ (~0xF & 0xF) = 0x0
print(f"Expected: 0x0")
# Output the result in hexadecimal format
print(f"Result: {hex(result)}" + "\n")

# Test 2
# All bits set to 1 for first input, all bits set to 0 for second input, all bits set to 1 for third input
print("Test 2")
result = Ch(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)
print(f"Expected: 0x0")
print(f"Result: {hex(result)}" + "\n")

# Test 3
# Alternating bits for first two inputs, all bits set to 1 for third input
print("Test 3")
result = Ch(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF)
print(f"Expected: 0x55555555")
print(f"Result: {hex(result)}" + "\n")


Testing Choice Function with hexadecimal inputs:
Expected: 0x0
Result: 0x0

Test 2
Expected: 0x0
Result: 0x0

Test 3
Expected: 0x55555555
Result: 0x55555555



In [12]:
# Majority Function Test with hexadecimal inputs
print("\nTesting Majority Function with hexadecimal inputs:")
# Test with hexadecimal 0xF which is 15 in decimal
result = Maj(0xF, 0xF, 0x0)
# Expected result should be 0xF as (0xF & 0xF) ^ (0xF & 0x0) ^ (0xF & 0x0) = 0xF
print(f"Expected: 0xf")
# Output the result in hexadecimal format
print(f"Result: {hex(result)}" + "\n")

# Test 2
# All bits set to 1 for the first and third inputs, all bits set to 0 for the second input
print("Test 2")
result = Maj(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")

# Test 3
# Alternating bits for first two inputs, all bits set to 1 for third input
print("Test 3")
result = Maj(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")


Testing Majority Function with hexadecimal inputs:
Expected: 0xf
Result: 0xf

Test 2
Expected: 0xffffffff
Result: 0xffffffff

Test 3
Expected: 0xffffffff
Result: 0xffffffff



In [13]:
# All Sigma Functions Tests
# Sigma0 (E0) Function Test with hexadecimal input
print("\nTesting Sigma0 (E0) Function")
result = Sigma0(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# Sigma1 (E1) Function Test
print("Testing Sigma1 (E1) Function")
result = Sigma1(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# sigma0 (o0) Function Test
print("Testing sigma0 (o0) Function")
result = sigma0(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# sigma1 (o1) Function Test
print("Testing sigma1 (o1) Function")
result = sigma1(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")


Testing Sigma0 (E0) Function
Result: 0xffffffff

Testing Sigma1 (E1) Function
Result: 0xffffffff

Testing sigma0 (o0) Function
Result: 0x1fffffff

Testing sigma1 (o1) Function
Result: 0x3fffff



# Problem 2: Fractional Parts of Cube Roots
## Overview
From the Secure Hash Standard PDF on page 11, we will use numpy to calculate the constants used in the SHA-224 & SHA-256 algorithms.
These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.

### Implementation Steps:
1. Create a `primes(n)` function that generates the first n prime numbers.
2. Create a function to calculate the cube root of the first 64 primes.
3. Extract the fractional part and convert to 32-bit integers.
4. Format and display results in hexadecimal.
5. Verify against the Secure Hash Standard.

### Approach
The function to generate the prime numbers uses an iterative approach. When given a number, checks each number up to that limit by testing if it has any divisions other than 1 and the number itself.

The Sieve of Eratosthenes is another approach to find prime numbers of a specific number. It will mark the multiples of each of the prime number starting from 2, leaving only prime numbers unmarked.

Since we are looking for the first 64 prime numbers I am using the iterative approach.

In [14]:
# Function to generate prime numbers
def primes(n):
    """
    Generate a `n` list of prime numbers.

    Parameters:
        n (int): The number of prime numbers to generate.

    Returns:
        list: A list of prime numbers up to `n`.
    """
    primeList = []

    num = 2

    # Generate prime numbers until we have `n` primes 
    while len(primeList) < n:
        isPrime = True

        # Check divisibility from 2 to the square root of num
        for i in range(2, int(np.sqrt(num)) + 1):
            if (num % i) == 0:
                isPrime = False
                break
            
        # If the number is prime, add it to the list
        if isPrime:
            primeList.append(num)
        num += 1

    return primeList

# Test usage
print("Generate the first 10 prime numbers:")
prime_numbers = primes(10)
print(prime_numbers)

Generate the first 10 prime numbers:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [15]:
# Function to Calculate the Cube Root of the prime numbers
def cube_root(primes):
    """
    This function calculates the cube root of the prime numbers

    Parameters:
        prime (list): A list of prime numbers.

    Returns:
        list: A list of cube roots of the given prime numbers.
    """
    cube_root_list = []

    # Calculate the cube root for each prime number
    for p in primes:
        # Use Python's exponentiation operator to calculate the cube root
        cube_root = p ** (1/3)
        cube_root_list.append(cube_root)
        
    return cube_root_list

# Test usage
print("\nCube roots of the first 10 prime numbers:")
cube_roots_list = cube_root(prime_numbers)
# print(cube_roots_list)

# Print the cube root values in rows of 4
for i in range(0, len(cube_roots_list), 4):
    print(cube_roots_list[i:i+4])


Cube roots of the first 10 prime numbers:
[1.2599210498948732, 1.4422495703074083, 1.7099759466766968, 1.912931182772389]
[2.2239800905693152, 2.3513346877207573, 2.571281590658235, 2.668401648721945]
[2.8438669798515654, 3.072316825685847]


In [16]:
# Function to extract the fractional part
def fractional_parts(cube_roots):
    """
    This function extracts the fractional part of the cube roots.

    Parameters:
        cube_roots (list): A list of cube roots.

    Returns:
        list: A list of fractional parts of the given cube roots.
    """
    fractional_list = []

    # Extract the fractional part for each cube root
    for root in cube_roots:
        # Subtract the integer part from the root to get the fractional part
        fractional_part = root - int(root)
        fractional_list.append(fractional_part)
        
    return fractional_list

# Test usage
print("\nFractional parts of the cube roots:")
fractional_parts_list = fractional_parts(cube_roots_list)
# print(fractional_parts_list)

# Print the fractional parts of the cube roots in rows of 4
for i in range(0, len(fractional_parts_list), 4):
    print(fractional_parts_list[i:i+4])


Fractional parts of the cube roots:
[0.2599210498948732, 0.4422495703074083, 0.7099759466766968, 0.9129311827723889]
[0.22398009056931523, 0.35133468772075727, 0.5712815906582351, 0.6684016487219449]
[0.8438669798515654, 0.07231682568584707]


In [17]:
# Function to extract 32 bits from the fractional parts
def extract_32bits(fractional_parts):
    """
    Extract the first 32 bits of the fractional part.
    
    Parameters:
        fractional_part (list): A list of fractional values (each between 0 and 1).
    
    Returns:
        int: The 32-bit integer value.
    """
    bits_list = []

    # Extract 32 bits from each fractional part
    for frac in fractional_parts:
        # Scale the fractional part to the range of a 32-bit integer
        scaled_value = int(frac * (2**32))
        bits_list.append(scaled_value)
        
    return bits_list

# Test usage
print("\nExtracted 32 bits from the fractional parts:")
extracted_bits_list = extract_32bits(fractional_parts_list)
print(extracted_bits_list)



Extracted 32 bits from the fractional parts:
[1116352408, 1899447441, 3049323471, 3921009573, 961987163, 1508970993, 2453635748, 2870763221, 3624381080, 310598401]


In [18]:
# Function to convert the extracted 32-bits to hexadecimal
def convert_to_hexadecimal(bits_value):
    """
    Convert the 32-bit integer to hexadecimal format.
    
    Parameters:
        bits_value (int): The 32-bit integer
    
    Returns:
        str: Hexadecimal representation
    """
    hex_list = []

    # Convert each 32-bit integer to hexadecimal format
    for value in bits_value:
        # Format as 8-character hexadecimal with leading zeros
        hex_value = f"0x{value:08x}"  
        hex_list.append(hex_value)
        
    return hex_list

# Test usage
print("\nConverted 32 bits to hexadecimal:")
hexadecimal_list = convert_to_hexadecimal(extracted_bits_list)
print(hexadecimal_list)


Converted 32 bits to hexadecimal:
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01']


In [19]:
# Problem 2 results with the first 64 prime numbers
# Sequence of operations
first_64_primes = primes(64)
cube_roots_64 = cube_root(first_64_primes)
fractional_parts_64 = fractional_parts(cube_roots_64)
extracted_bits_64 = extract_32bits(fractional_parts_64)
hexadecimal_64 = convert_to_hexadecimal(extracted_bits_64)

print("SHA-224 & 256 K Constants of the first 64 prime numbers:")
# This will print 8 hexadecimal values per line
for i in range(0, len(hexadecimal_64), 8):
    print(hexadecimal_64[i:i+8])

SHA-224 & 256 K Constants of the first 64 prime numbers:
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']
['0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174']
['0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da']
['0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967']
['0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85']
['0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070']
['0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3']
['0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']


In [20]:
# Comparing generated K constants with the official SHA-224 & 256 K constants
# Manually copied from page 11 of the Secure Hash Standard PDF, see References for the link
officialKConstants = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5, 0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174, 0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da, 0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967, 0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85, 0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070, 0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3, 0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

# Convert to string for comparison
# Takes the official K constants and converts them to strings in hexadecimal format for easier comparison with the generated hexadecimal values
officialKConstString = [f"0x{value:08x}" for value in officialKConstants]

# Print both official and generated K constants in a row
print("\nOfficial SHA-224 & 256 K Constants:")
for i in range(0, len(officialKConstString), 8):
    print(officialKConstString[i:i+8])
    
print("\nGenerated K Constants:")
for i in range(0, len(hexadecimal_64), 8):
    print(hexadecimal_64[i:i+8])

# Compare my generated K constants with the official SHA-224 & 256 K constants
print("\nComparing generated K constants with the official SHA-224 & 256 K constants:")
if hexadecimal_64 == officialKConstString:
    print("The generated K constants match the official SHA-224 & 256 K constants.")
else:
    print("The generated K constants do NOT match the official SHA-224 & 256 K constants.")


Official SHA-224 & 256 K Constants:
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']
['0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174']
['0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da']
['0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967']
['0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85']
['0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070']
['0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3']
['0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']

Generated K Constants:
['0x428a2f98', '0x71374491', '0xb5c

# Problem 3: Padding

## Overview
Create a generator function `block_parse(msg)` that process messages according to sections 5.1.1 and 5.2.1 of the Secure Hash Standard PDF.
The function should accept a bytes object called `msg`. The message should be padded and ensure it's in a multiple of 512 or 1024 bits.

### Implementation Steps
1. Check the padding requirements from the Secure Hash Standard PDF (Sections 5.1.1 & 5.2.1).
2. Calculate the padding needed for the message.
3. Construct the padded message -> original message + padding bits + length.
4. Create the generator function yielding 512-bit blocks.
5. Test with different various message lengths e.g. minimum, average and near maximum. 

### Approach
The SHA-256 hashing algorithm processes messages in fixed 512-bit (64-byte) blocks. 
The actual messages can be any length, but we will need to pad the remaing length of the message so we can then set it up for hashing.

**Padding Structure from Section 5.1.1**

When padding we follow a specific format:
1. **The '1' bit**: We append `0x80` (binary: 10000000) to mark where the original message ends
2. **Zero padding**: We add zero bytes until we're 64 bits (8 bytes) short of a 512-bit boundary
3. **Length field**: The final 64 bits store the original message length in bits (big-endian format)

This means we always leave room for the 64-bit length at the end, which is why we pad to 448 bits (mod 512) rather than 512.

**Use of a Generator Function**

We use a generator function (`yield`) rather than returning a list of blocks because:
- **Memory efficiency**: For large messages, we don't need to store all blocks in memory at once
- **Lazy evaluation**: Blocks are only created when needed
- **Cleaner iteration**: The calling code can simply loop through blocks without managing indices

**Edge Cases Considered**

- **Empty message**: Still produces one block (just padding and length)
- **Messages near block boundaries**: May require an extra block if there isn't room for padding
- **Long messages**: The generator handles any number of blocks efficiently


In [21]:
# Calculate and pad the message according to SHA-256 specification
def calculate_padding(message):
    """
    This function calculates the padding for a given message according to the SHA-256 specification.

    1. Append a single '1' bit to the message.
    2. Append '0' bits until the length of the message is congruent to 448 mod 512.
    3. Append the original length of the message as a 64-bit big-endian integer.

    Parameters:
        message (bytes): The original message in bytes.

    Returns:
        bytes: The padded message in multiple of 512 bits / 64 bytes.
    """
    # Calculate the original length of the message in bits
    original_byte_len = len(message)
    # Convert byte length to bit length
    original_bit_len = original_byte_len * 8

    # Append the bit '1' to the message, markes the end of the original message
    padded = message + b'\x80'

    # Append '0' bits until the length of the message is congruent to 448 mod 512
    while (len(padded) * 8) % 512 != 448:
        padded += b'\x00'

    # Append the original length as a 64-bit big-endian integer
    padded += original_bit_len.to_bytes(8, byteorder='big')

    return padded

# Test usage
print("\nCalculating padding for the message 'abc':")
# Input a test message in bytes using the b prefix to indicate a byte string
test_msg = b'abc'
result = calculate_padding(test_msg)

# Check the length
print(f"Padded message length: {len(result)} bytes")
print(f"Padded message length: {len(result) * 8} bits")




Calculating padding for the message 'abc':
Padded message length: 64 bytes
Padded message length: 512 bits


In [22]:
# Generator function to process messages
def block_parse(msg):
    """
    This generator function processes the input message in 512-bit (64-byte) blocks.

    Parameters:
        msg (bytes): The padded message in bytes.

    Yields:
        bytes: 64-byte blocks of the message.
    """
    # Loop through the message in steps of 64 bytes
    for i in range(0, len(msg), 64):
        yield msg[i:i+64]

In [23]:
# Test usage
print("\nTesting with a short message 'abc':")
test_msg = b'abc'
padded_msg = calculate_padding(test_msg)

# Process and print each 64-byte block
for i, block in enumerate(block_parse(padded_msg)):
    print(f"\nBlock {i}:")
    print(block)
    print(f"Block {i} length: {len(block)} bytes")

# Test with an empty message
print("\nTesting with an empty message:")
empty_msg = b''
padded_empty_msg = calculate_padding(empty_msg)

for i, block in enumerate(block_parse(padded_empty_msg)):
    print(f"\nBlock {i}:")
    print(block)
    print(f"Block {i} length: {len(block)} bytes")

# Test with a longer message
print("\nTesting with a longer message:")
long_msg = b'a' * 600  # 600 bytes of 'a'
padded_long_msg = calculate_padding(long_msg)

for i, block in enumerate(block_parse(padded_long_msg)):
    print(f"\nBlock {i}:")
    print(block)
    print(f"Block {i} length: {len(block)} bytes")



Testing with a short message 'abc':

Block 0:
b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18'
Block 0 length: 64 bytes

Testing with an empty message:

Block 0:
b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Block 0 length: 64 bytes

Testing with a longer message:

Block 0:
b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
Block 0 length: 64 bytes

Block 1:
b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
Block 1 length: 64 bytes

Block 2:
b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
Block 2 length: 64 bytes

Block 3:

# Problem 4: Hashes

## Overview
Write a function `hash(current, block)` that will take the current hash value and return the next hash state. This is according to section 6.2.2 of the SHA-256 specification.

### Implementation Steps
1. Define and use the 64 SHA-256 K constants.
2. Use previous functions and prepare the message for scheduling.
3. Initialise the values form the current hash and assign them to variables.
4. Loop through the message for each of the K constants.
5. Calculate the next Hash value and output the new message after processing.

### Approach
**The Compression Function (Section 6.2.2)**

The SHA-256 hash is computed by processing each 512-bit block through a compression function. This function takes the current hash state (8 words) and a message block, then outputs the next hash state.

**Message Schedule**

Before compression, each 64-byte block is expanded into 64 words (W₀ to W₆₃):
- The first 16 words come directly from the block (split into 32-bit chunks)
- The remaining 48 words are derived using the lowercase sigma functions from Problem 1

**The 64 Rounds**

Each round uses:
- The K constants from Problem 2
- The bitwise functions (Ch, Maj, Σ₀, Σ₁) from Problem 1
- Modular addition to update the 8 working variables (a-h)

After all 64 rounds, the working variables are added back to the current hash state to produce the next hash value.

In [24]:
# K Constants for SHA-224 & 256, already defined in problem 2 called officialKConstants
# Convert the K constants to numpy uint32 format
K = np.array(officialKConstants, dtype=np.uint32)
print("The first 8 K constants:")
print(K[:8])
print(f"Total number of K constants: {len(K)}")

The first 8 K constants:
[1116352408 1899447441 3049323471 3921009573  961987163 1508970993
 2453635748 2870763221]
Total number of K constants: 64


In [25]:
# Message Schedule Function
def message_schedule(block):
    """
    This function creates a message schedule array for a given 512-bit block.
    It implements the message schedule preparation as specified in the SHA-256 standard (Section 6.2.2).

    Parameters:
        block (bytes): A 64-byte (512-bit) block of the message.

    Returns:
        np.ndarray: An array of 64 32-bit unsigned integers representing the message schedule.
    """
    # Initialise the message schedule array with zeros.
    W = np.zeros(64, dtype=np.uint32)

    # Step 1: Break the block into sixteen 32-bit big-endian words.
    for t in range(16):
        W[t] = np.uint32(int.from_bytes(block[t*4:(t*4)+4], byteorder='big'))

    # Step 2: Extend the first 16 words into the remaining 48 words of the message schedule array.
    # Use the formula in the SHA-256 specification at 6.2.2 to process the message.
    for t in range(16, 64):
        # Calculate the lowercase sigma0 function
        s0 = sigma0(W[t - 15]) # lowercase sigma0
        # Calculate lowercase sigma1
        s1 = sigma1(W[t - 2]) # lowercase sigma1
        # Calculate W[t] using the specified formula and ensure it is a 32-bit unsigned integer. The & 0xFFFFFFFF ensures that we only keep the lower 32 bits.
        W[t] = np.uint32((W[t - 16] + s0 + W[t - 7] + s1) & 0xFFFFFFFF)

    return W

In [26]:
# Hash function to calculate the next hash state
def hash(current, block):
    """
    Calculate the next hash state given the current hash state and a message block.

    Parameters:
        current (list): The current hash state as a list of 8 32-bit unsigned integers.
        block (bytes): A 64-byte (512-bit) block of the message.

    Returns:
        list: The updated hash state as a list of 8 32-bit unsigned integers.
    """
    # Defined K constants
    K = np.array(officialKConstants, dtype=np.uint32)

    # Prepare the message schedule
    W = message_schedule(block)

    # Initalise the 8 working variables form the current hash value
    a = current[0]
    b = current[1]
    c = current[2]
    d = current[3]
    e = current[4]
    f = current[5]
    g = current[6]
    h = current[7]

    # Main compression loop
    for t in range(64):
        # Calculate T1 and T2 using the SHA-256 functions
        T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
        T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
        
        # Update the working variables
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)

    # Update the hash state with the new values
    new_hash = [
        np.uint32(current[0] + a),
        np.uint32(current[1] + b),
        np.uint32(current[2] + c),
        np.uint32(current[3] + d),
        np.uint32(current[4] + e),
        np.uint32(current[5] + f),
        np.uint32(current[6] + g),
        np.uint32(current[7] + h)
    ]
    
    return new_hash

In [27]:
# Test usage of the hash function
print("\nTesting the hash function with the message 'abc':")
test_msg = b'abc'
padded_msg = calculate_padding(test_msg)

# Process the padded message into 512-bit blocks
blocks = list(block_parse(padded_msg))

# Initial hash values for SHA-256
initial_hash = [ 0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19 ]

# Compute the hash over all blocks
current_hash = initial_hash

# Iterate over each block and update the hash state
for block in blocks:
    current_hash = hash(current_hash, block)

# Output the final hash value in hexadecimal format
hash_hex = [f"0x{value:08x}" for value in current_hash]
print(f"Computed hash for 'abc': {hash_hex}")


Testing the hash function with the message 'abc':
Computed hash for 'abc': ['0xba7816bf', '0x8f01cfea', '0x414140de', '0x5dae2223', '0xb00361a3', '0x96177a9c', '0xb410ff61', '0xf20015ad']


# Problem 5: Passwords


## Overview
We are provided with 3 common passwords that have been hashed with one pass of the SHA-256 algorithm. We need to 'crack'/identify the original passwords and find out their literal strings before they were encoded and hashed.
We will use a **Dictionary Attack**, where we will hash the most common passwords and compare them to our listed passwords until we can find a match. This type of attack works as many passwords are created with a small set of common words and patterns.
Following that we need to suggest security improvements to make password hashes more resistant to this type of attack.

Passwords to crack:
1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

### Implementation Steps:
1. Create a list of common passwords in a dictionary.
2. Use the hash function from Problem 4, to hash the passwords based on the SHA-256 algorithm.
3. Convert the target hashes into a searchable and comparable format.
4. Iterate through each hash and compare with hashed dictionary passwords:
   - Hash the password.
   - Convert the resulting hash to a hex string.
   - Compare the password from the dictionary to the 3 provided hashed passwords.
5. Report which passwords were found and not found in the dictionary.

### Approach
Use a brute-force dictionary attack:
- Iterate through the 3 hashed passwords to crack.
- For each one, we need to test against the passwords in the dictionary:
    - Convert them to bytes.
    - Hash using the hash function from problem 4.
    - Convert the computed hashes to a hex string format.
    - Compare them against the target hashes.
- Record the password when a match is found then move onto the next hashed password to crack.

In [28]:
# Hashed passwords to crack
hashed_passwords = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"]

In [29]:
# Function to hash the passwords combining functions from Problem 3 & 4
def hash_password(password):
    """
    Hash a password using the SHA-256 algorithm.

    Takes a plaintext password string and returns its SHA-256 hash in hexadecimal format.

    Parameters:
        password (str): The plaintext password to hash.

    Returns:
        str: The SHA-256 hash of the password in hexadecimal format.
    """
    # Encode the password to UTF-8 bytes and apply padding
    padded_msg = calculate_padding(password.encode('utf-8'))

    # Parse into blocks
    blocks = list(block_parse(padded_msg))

    # Initialise hash values
    initial_hash = [ 0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19 ]
    current_hash = initial_hash
    
    # Loop through blocks and process the hash compression function
    for block in blocks:
        current_hash = hash(current_hash, block)

    # Convert to hex string
    hash_hex = ''.join([f"{value:08x}" for value in current_hash])
    return hash_hex

In [30]:
# Test usage of the hash_password function
print("\nTesting the hash_password function:")
test_password = hash_password("abc")
print(f"Computed hash of 'abc': {test_password}")

expected_abc_hash = "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad"
print(f"Expected hash of 'abc': {expected_abc_hash}")
print(f"Match: {test_password == expected_abc_hash}")


Testing the hash_password function:
Computed hash of 'abc': ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Expected hash of 'abc': ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Match: True


In [ ]:
# Load passwords form text file
def load_passwords(file_path):
    """
    Load passwords from a text file.

    Parameters:
        file_path (str): The path to the text file containing passwords.
    Returns:
        list: A list of passwords.
    """
    passwords = []

    # Read the file and extract passwords
    try:
        # Open the file in read mode with UTF-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Strip whitespace and newline characters
                password = line.strip()

                # Ignore empty lines and comments
                if password and not password.startswith('#'):
                    passwords.append(password)

        # Output the number of loaded passwords            
        print(f"Loaded {len(passwords)} passwords from {file_path}")
        return passwords
    
    # Handle file not found error
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return []
    
    except Exception as e:
        print(f"An error occurred while loading passwords: {e}")
        return []

In [ ]:
# Load the passwords form the specified text file
password_file_path = '100k-most-used-passwords-NCSC.txt'

password_list = load_passwords(password_file_path)

# Print the first 5 passwords as a sample
print(f"Total passwords loaded: {len(password_list)}")
print(f"First 5 passwords: {password_list[:5]}")

Loaded 99818 passwords from 100k-most-used-passwords-NCSC.txt
Total passwords loaded: 99818
First 5 passwords: ['123456', '123456789', 'qwerty', 'password', '111111']


In [33]:
# Use all passwords for the dictionary attack
passwords_to_test = password_list

print(f"\n{'=' * 40}")
print(f"Testing {len(passwords_to_test)} passwords...")
print(f"{'=' * 40}\n")


Testing 99818 passwords...



In [ ]:
# Dictionary attack to find the hashed passwords
def dictionary_attack(hashed_passwords, password_list):
    """
    Perform a dictionary attack to find the plaintext passwords corresponding to the given hashed passwords.

    Parameters:
        hashed_passwords (list): A list of hashed passwords in hexadecimal format.
        password_list (list): A list of potential plaintext passwords.

    Returns:
        dict: A dictionary mapping found hashed passwords to their corresponding plaintext passwords.
    """
    # Dictionary to store found passwords
    found_passwords = {}

    # Loop through each password in the password list
    for password in password_list:
        # Hash the current password
        hashed = hash_password(password)

        # Check if the hashed password matches any of the target hashed passwords
        if hashed in hashed_passwords:
            found_passwords[hashed] = password
            print(f"Found match! Password: '{password}' => Hash: {hashed}")

            # Remove found hash from the list to speed up future checks
            hashed_passwords.remove(hashed)

        # If all hashes have been found, break early
        if not hashed_passwords:
            break

    return found_passwords

In [ ]:
# Crack the hashed passwords
results = dictionary_attack(hashed_passwords.copy(), passwords_to_test)

# Display the results
print(f"\n{'='*50}")
print("Attack Results:")
print(f"{'='*50}")

# Print each found password with its corresponding hash
for hashed, password in results.items():
    print(f"Hash: {hashed}")
    print(f"Password: '{password}'")
    print(f"Verification: {hash_password(password)}\n")

# Check if any hashes were not found
# If the number of results is less than the number of hashed passwords, some were not found
if len(results) < len(hashed_passwords):
    # Calculate how many were not found
    not_found = len(hashed_passwords) - len(results)
    print(f"Not found {not_found} hashed passwords.")
else:
    print("All hashed passwords were successfully cracked!")

Found match! Password: 'password' => Hash: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Found match! Password: 'cheese' => Hash: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Found match! Password: 'P@ssw0rd' => Hash: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

Attack Results:
Hash: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Password: 'password'
Verification: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8

Hash: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Password: 'cheese'
Verification: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34

Hash: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
Password: 'P@ssw0rd'
Verification: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

All hashed passwords were successfully cracked!


## Methodology
The 3 hashed passwords were created in a single pass of the SHA-256 algoritm.
I used a **Dictionary Attack** to crack the passwords, by computing hashes of common passwords and comparing them to the target hashes until a match is found.

### Process
1. We took a password from the NCSC password list (99,818) and hashed it using my own SHA-256 implementation (see Problems 3 & 4).
   
2. Each password was then padded and parsed according to the SHA-256 specification:
   - Encoded to UTF-8 bytes
   - Padded according to Section 5.1.1 in the PDF
   - Parsed into 512-bit blocks using the block_parse() generator function

3. Each block was processed using the SHA-256 compression function (Section 6.2.2):
   - The 7 bitwise functions (Parity, Ch, Maj, Sigma0, Sigma1, sigma0, sigma1)
   - The 64 round constants (K values from cube roots of primes)
   - The 8 initial hash values

4. We then compare the resulting 256-bit hash to the 3 target hashes. When we find a match, we have found the likely password.

### Why This Works:
The dictionary attack succeeds because:
- **SHA-256 is predetermined**: the same text input will produce the same hash output, it makes it easy to compare
- **Passwords were common**: all three passwords appeared in the NCSC list of passwords
- **Modern computational speed**: hashing the 99,818 passwords only took a few moments to process on relatively standard hardware, make this search possible

### Solutions
This attack only exploits  **common passwords**. These following measures can help prevent or slow down this type of attack:

1. **Longer passwords**: Use passwords with at least 12+ characters, mixing uppercase, lowercase, numbers and special characters. Avoid common words, patterens and keyboard sequences ("12345" or "asdfgh"). Doing this would make dictionary attacks ineffective as random, longer passwords are unlikely to appear in a password list (unless exposed in a data breach).
   
2. **Salting**: Add a random `salt` (random bytes) to the password before hashing it. This ensures each password produces a unique hash, even if two users have the same password. It also increases computational cost as attackers would need to test each possible password with against every possible salt value, exponentially increasing the work required.
   
3. **Slow Hashing Algorithm**: Use more password specific algorithms such as bcrypt, scrypt or Argon2, these intentionally slow down hasing (taking up to 100ms per hash instead of microseconds). So in this case, testing against 99,818 passwords will take exponentially longer (days or weeks instead of minutes), making this brute force approach very impractical.


# References

### Primary Source
- **NIST FIPS 180-4: Secure Hash Standard** - The official specification for SHA-256 and related hash algorithms. Available at: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

### Implementation References
- **Prime Number Generation** - Finxter article on different approaches to generating prime numbers in Python: https://blog.finxter.com/5-best-ways-to-create-a-list-of-prime-numbers-in-python/
- **Python Generator Functions** - Real Python guide on creating and using generator functions: https://realpython.com/introduction-to-python-generators/

### Hashing Concepts
- **What is Hashing** - Codecademy article explaining the fundamentals of hashing: https://www.codecademy.com/resources/blog/what-is-hashing
- **Hash Algorithms Overview** - Visual explanation of how hash algorithms work: https://jdspugh.github.io/2023/04/06/hash-algorithms.html
- **Why SHA Messages are Padded** - Crypto Stack Exchange discussion on the purpose of padding in SHA algorithms: https://crypto.stackexchange.com/questions/2753/in-the-sha-hash-algorithm-why-is-the-message-always-padded


### Password Security Resources
- **NCSC Password List** - The National Cyber Security Centre's list of 100,000 common passwords, originally sourced from 'Have I Been Pwned'. Retrieved from: https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/100k-most-used-passwords-NCSC.txt
- **NCSC Password Guidance** - Article on configuring NCSC password lists in Active Directory: https://specopssoft.com/blog/configure-ncsc-password-list-in-ad/
- **Password Hashing, Salting and Algorithms** - Authgear article explaining password hashing best practices and why salting is important: https://www.authgear.com/post/password-hashing-salting-function-and-algorithm-explained

## Acknowledgements
Classmates for general discussion for the assessment and providing some feedback on the notebook.



# Conclusion

This notebook implements a version of the SHA-256 hashing algorithm, following the NIST Secure Hash Standard (FIPS 180-4). The implementation demonstrates:

1. **Bitwise Operations (Problem 1)**: All seven bitwise functions (Parity, Ch, Maj, Σ₀, Σ₁, σ₀, σ₁) were implemented using NumPy's uint32 type to ensure correct 32-bit behaviour.

2. **Constant Generation (Problem 2)**: The K constants were successfully derived from the cube roots of the first 64 prime numbers, matching the official specification.

3. **Message Padding (Problem 3)**: A generator function was created to properly pad and parse messages into 512-bit blocks according to the standard.

4. **Hash Computation (Problem 4)**: The compression function was implemented following Section 6.2.2, producing correct SHA-256 hashes.

5. **Password Cracking (Problem 5)**: Using a dictionary attack with approximately 100,000 common passwords, all three target hashes were successfully cracked, demonstrating both the effectiveness of the implementation and the vulnerability of simple password hashing.

## Key Takeaways
- SHA-256 is deterministic: the same input always produces the same output
- The algorithm relies on careful bit manipulation and modular arithmetic
- Simple password hashing (without salting) is vulnerable to dictionary attacks
- Modern password storage should use algorithms like bcrypt, scrypt, or Argon2

# End